# 📊 Notebook: Event Impact Modeling
**Task:** 3 – Build the Event-Indicator Association Matrix  
**Objective:** To quantify how specific events (Policies, Product Launches, Infrastructure) shift the trajectory of financial inclusion indicators.

### 🧠 The Logic: The "Event-Augmented" Forecast
Standard time-series models (ARIMA/Prophet) rely on long history. We don't have that (only 5 data points).
Instead, we will use a **Structural Scenario Model**:

$$ Y_{t} = Y_{trend} + \sum (Event_{impact} \times Decay_{factor}) $$

Where:
*   **$Y_{trend}$**: The natural organic growth (e.g., population growth, gradual adoption).
*   **$Event_{impact}$**: A one-time or permanent shift caused by a shock (e.g., M-Pesa Launch).
*   **$Decay_{factor}$**: How long it takes for the event to reach full potential (Lag).

In this notebook, we calculate the values for **$Event_{impact}$**.

---

## 1. Setup & Data Loading

In [11]:
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta

# Add src to path for imports
sys.path.append(os.path.abspath('..'))

# Visual Config
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# 1. Load Processed Data (Observations + Events)
df = pd.read_csv('../data/processed/ethiopia_fi_enriched.csv', parse_dates=['observation_date'])

# 2. Load Impact Links (The "Logic" Sheet)
# We need to reload the raw Excel to get the Impact_sheet, or load a CSV if you saved it separately.
# Assuming we load from raw for this specific sheet as it wasn't modified in Task 1, 
# OR if you saved it in Task 1, load from processed. 
# Here we stick to the raw loader for the specific sheet to be safe.
try:
    from src.loader import load_data
    impact_df = load_data('../data/raw/ethiopia_fi_unified_data.xlsx', sheet_name='Impact_sheet')
except Exception as e:
    # Fallback if src not found (running directly)
    print("⚠️ Loader not found, using direct pandas read")
    impact_df = pd.read_excel('../data/raw/ethiopia_fi_unified_data.xlsx', sheet_name='Impact_sheet')

# Filter main DF into Events and Observations
events_df = df[df['record_type'] == 'event'].copy()
observations_df = df[df['record_type'] == 'observation'].copy()

print(f"✅ Loaded {len(events_df)} Events and {len(impact_df)} Impact Links.")

✅ Loaded 12 Events and 14 Impact Links.


## 2. Build the Event-Indicator Matrix
We need to join the `Events` (Which *Date*?) with the `Impact Links` (Which *Effect*?).

In [12]:
# 1. Merge Impact Links with Event Details
# We join on 'parent_id' (Impact Sheet) == 'record_id' (Data Sheet)
full_impact_df = pd.merge(
    impact_df,
    events_df[['record_id', 'event_name', 'observation_date', 'category']],
    left_on='parent_id',
    right_on='record_id',
    how='left'
)

# 2. Clean up columns for the Matrix View
matrix_cols = [
    'event_name', 'category', 'observation_date', 
    'indicator_code', 'impact_direction', 
    'impact_magnitude', 'impact_estimate', 'lag_months', 'notes'
]

# Filter for rows where we actually have a match
impact_matrix = full_impact_df[matrix_cols].dropna(subset=['event_name'])

print("Sample Impact Matrix:")
display(impact_matrix.head(3))

KeyError: "['event_name'] not in index"

### 3.1 Analyzing the Conversion Rate
Telebirr reports ~47 Million users. Findex reports 49% of adults.
If the population of adults (15+) is ~60 Million:
*   49% = ~29.4 Million people.
*   Telebirr claims 47 Million users.

**Insight:** There are more Telebirr accounts than total unique financial accounts. This proves that **1 Telebirr User $\neq$ 1 New Included Person**. Most Telebirr users likely already had bank accounts, or have the app but don't use it enough to count for Findex.

We will calculate a **Conservative Impact Coefficient**.
Hypothesis: Telebirr contributed to the 3pp growth between 2021 (46%) and 2024 (49%).
*   Growth: +3% of 60M adults = 1.8M new unique users.
*   Telebirr Users added: ~40M.
*   **Conversion Ratio**: 1.8M / 40M $\approx$ **4.5%**.

**Model Assumption**: For every 100 new Mobile Money registrations, Account Ownership (Access) increases by roughly **4.5 people**.

In [9]:
# Define our Derived Parameters
ADULT_POPULATION = 60_000_000 # Estimate
TELEBIRR_USERS_ADDED = 40_000_000 # Approx 2021-2024
FINDEX_USERS_ADDED = ADULT_POPULATION * 0.03 # 3% growth

CONVERSION_RATE = FINDEX_USERS_ADDED / TELEBIRR_USERS_ADDED
print(f"Calculated Mobile-to-Findex Conversion Rate: {CONVERSION_RATE:.4f} ({CONVERSION_RATE*100:.1f}%)")

Calculated Mobile-to-Findex Conversion Rate: 0.0450 (4.5%)


## 4. Building the Association Matrix

We now map this logic to other events.
*   **Telebirr**: Validated (Low conversion impact on Access, High impact on Usage).
*   **M-Pesa**: Market entrant Aug 2023. We assume similar physics to Telebirr.
*   **NBE Liberalization**: Policy change. Enabler for M-Pesa.

We will create a matrix merging the raw `events` with `Impact_sheet` and filling in the estimates.

In [10]:
# 1. Merge Events and Impacts
event_impact_df = pd.merge(
    events[['event_id', 'event_name', 'event_date', 'category']],
    impacts_raw,
    left_on='event_id',
    right_on='parent_id',
    how='left'
)

# 2. Refine Estimates based on our Calculation
# We update the 'impact_magnitude' for Mobile Money launches based on the conversion rate
# If Magnitude is missing (NaN), we impute it.

def estimate_magnitude(row):
    # If we already have a manual estimate in the excel, keep it
    if pd.notnull(row['impact_magnitude']):
        return row['impact_magnitude']
    
    # Logic for M-Pesa (Product Launch)
    if 'M-Pesa' in row['event_name'] and row['related_indicator'] == 'ACC_OWNERSHIP':
        # Assumption: Safaricom targets 10M users.
        # Impact = 10M * Conversion_Rate / Adult_Pop
        target_users = 10_000_000
        impact_pp = (target_users * CONVERSION_RATE) / ADULT_POPULATION * 100
        return impact_pp # Returns percentage points (e.g., 0.75)

    return 0.0

event_impact_df['calculated_impact'] = event_impact_df.apply(estimate_magnitude, axis=1)

# 3. Create the Matrix View
association_matrix = event_impact_df.pivot_table(
    index=['event_date','event_name'],
    columns='related_indicator',
    values='calculated_impact'
).fillna(0)

print("Association Matrix (Estimated Impact in Percentage Points):")
display(association_matrix)

KeyError: "['event_id', 'event_name', 'event_date'] not in index"